## Armado de la base principal

Se propondrá como objetivo del trabajo otorgar una propuesta para la legislatura del gobierno de la ciudad.  Para tal fin se recurre a las páginas de las comunas en FB, y utilizar los comentarios como base de análisis.

__________________

In [1]:
import glob, os    
import pandas as pd
import string
import re

Armo una sola base para luego realizar la limpieza

In [2]:
allFiles = glob.glob("comuna*.csv")
base = pd.DataFrame()
list_ = []
for file_ in allFiles:
    df = pd.read_csv(file_,index_col=None, header=0, encoding="latin1")
    list_.append(df)
base = pd.concat(list_)

In [3]:
base.head()

File        _ - id         _ - posts - data - _ - id  \
0  comuna01.json  1.291073e+14  129107250440944_1945177118833939   
1  comuna01.json           NaN  129107250440944_1944242785594039   
2  comuna01.json           NaN  129107250440944_1943221939029457   
3  comuna01.json           NaN  129107250440944_1941172825901035   
4  comuna01.json           NaN  129107250440944_1937861752898809   

                      _ - posts - data - _ - message  \
0  Â¡Hoy es su dÃ­a y su mejor regalo es pasar mÃ...   
1  Â¡Feliz Aniversario San Telmo! í ½í²í ¼í¾\n\...   
2  En tus manos estÃ¡ la esperanza de vida para l...   
3  La Ciudad cuenta con 204 #SenderosEscolares pr...   
4  í ¼í·¦í ¼í··í ¼í·¦í ¼í·· Â¡Feliz #DÃ­aDeLaPatr...   

  _ - posts - data - _ - comments - data - _ - id  \
0                                             NaN   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4               1937861752898809_1941151222569862   

  _ - posts - data - _ - comments - data - _ - message  \
0                                                NaN     
1                                                NaN     
2                                                NaN     
3                                                NaN     
4  Buscamos Adela, la vieron hoy en suipacha y Bm...     

  _ - posts - data - _ - comments - data - _ - created_time  \
0                                                NaN          
1                                                NaN          
2                                                NaN          
3                                                NaN          
4                           2018-05-28T19:10:07+0000          

   _ - posts - data - _ - comments - data - _ - from - id  \
0                                                NaN        
1                                                NaN        
2                                                NaN        
3                                                NaN        
4                                                NaN        

  _ - posts - data - _ - comments - data - _ - from - name  \
0                                                NaN         
1                                                NaN         
2                                                NaN         
3                                                NaN         
4                                                NaN         

  _ - posts - data - _ - comments - paging - next  \
0                                             NaN   
1                                             NaN   
2                                             NaN   
3                                             NaN   
4                                             NaN   

  _ - posts - data - _ - comments - paging - cursors - after  \
0                                                NaN           
1                                                NaN           
2                                                NaN           
3                                                NaN           
4                                             MQZDZD           

  _ - posts - data - _ - comments - paging - cursors - before  \
0                                                NaN            
1                                                NaN            
2                                                NaN            
3                                                NaN            
4                                             MQZDZD            

                           _ - posts - paging - next  \
0  https://graph.facebook.com/v3.0/12910725044094...   
1                                                NaN   
2                                                NaN   
3                                                NaN   
4                                                NaN   

                _ - posts - paging - cursors - 

Renombro variables y elimino las variables que no me interesan

In [4]:
base.columns

Index(['File', '_ - id', '_ - posts - data - _ - id',
       '_ - posts - data - _ - message',
       '_ - posts - data - _ - comments - data - _ - id',
       '_ - posts - data - _ - comments - data - _ - message',
       '_ - posts - data - _ - comments - data - _ - created_time',
       '_ - posts - data - _ - comments - data - _ - from - id',
       '_ - posts - data - _ - comments - data - _ - from - name',
       '_ - posts - data - _ - comments - paging - next',
       '_ - posts - data - _ - comments - paging - cursors - after',
       '_ - posts - data - _ - comments - paging - cursors - before',
       '_ - posts - paging - next', '_ - posts - paging - cursors - after',
       '_ - posts - paging - cursors - before'],
      dtype='object')

In [5]:
base.rename(columns={'File':'comuna','_ - posts - data - _ - message':'post',
                       '_ - posts - data - _ - comments - data - _ - message':'message',
                    '_ - posts - data - _ - comments - data - _ - created_time':'time'}, inplace = True)


df = base[['comuna','post','message','time']]


A través de la visualización y comparación con la página se observa que cuando el comentario remite a un post, el mismo no se repite sino que en el csv figura nulo.  Con lo cual se completará los missing de los mismos.

In [6]:
df['post'].fillna(method='ffill', inplace=True)

/Users/hernangoldenberg/anaconda3/lib/python3.5/site-packages/pandas/core/generic.py:5430: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


Al crear la base se mantuvo los índices de cada base, se procederá a la reindexación.

In [7]:
df = df.reset_index(drop=True)

Se eliminará todas las filas que no poseen comentarios.  Es decir, el json se creó en base a los post, con lo cual aquellos post que no poseen comentarios serán eliminados.

In [8]:
df = df[df['message'].notna()]

Se procederá a la limpieza de las variables:

1- Se eliminará el ".json" de los datos de la variable **comuna**.

2- Limpiaremos la variable **time** y la volveremos una variable fecha, con sus propiedades.

3- Tanto a la variable **post** y **message** se trabajará con todo aquello sea ruido, como por ejemplo la acentuación y la ñ que es leída como un caracter no comprendido por python.

** Comuna **

In [9]:
df['comuna'] = df['comuna'].str.replace('.json', "")

In [10]:
df['comuna'].value_counts()

comuna11    6856
comuna10    5344
comuna09    4934
comuna08    3976
comuna07    3331
comuna12    3309
comuna13    3217
comuna06    2986
comuna05    2829
comuna03    2642
comuna02    2270
comuna15    2002
comuna01    1621
comuna14    1484
comuna04    1447
Name: comuna, dtype: int64

** Time **

In [11]:
df['date'] = pd.to_datetime(df['time'])
del df['time']

**Post** & **Comentarios**

Primero volveremos todo a minúscula los textos.

In [12]:
df = df.apply(lambda x: x.astype(str).str.lower())

In [13]:
#a
df['post'] = df['post'].str.replace(u"ã¡", "a")
df['message'] = df['message'].str.replace(u"ã¡", "a")

#e
df['post'] = df['post'].str.replace(u"ã©", "e")
df['message'] = df['message'].str.replace(u"ã©", "e")

#i
df['post'] = df['post'].str.replace(u"ã­", "i")
df['message'] = df['message'].str.replace(u"ã­", "i")

#o
df['post'] = df['post'].str.replace(u"ã³", "o")
df['message'] = df['message'].str.replace(u"ã³", "o")

#u
df['post'] = df['post'].str.replace(u"ãº", "u")
df['message'] = df['message'].str.replace(u"ãº", "u")

#ñ
df['post'] = df['post'].str.replace(u"ã±","ñ")
df['message'] = df['message'].str.replace(u"ã±","ñ")

In [14]:
df = df.reset_index(drop=True)
df.to_csv('face_comun.csv') # Guardamos la base en .csv

Se observa que existe páginas web declaradas en el post y en comentarios.  Se procederá a eliminar la ruta.

In [15]:
# Página web

df['post'] = df['post'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)
df['message'] = df['message'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

Se eliminará todos los caracteres que no son texto.

In [16]:
from nltk.stem.wordnet import WordNetLemmatizer

def elimin(t):
    return re.sub("[^a-z' ]+", '', t)

# Idem punto anterior, se excluyen signos de puntuacion 
exclude = set(string.punctuation) 
print(len(exclude))
print(exclude)

# Lemmatizacion de las palabras 
lemma = WordNetLemmatizer()

def clean(doc):
    punc_free = ''.join(ch for ch in doc if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

32
{'`', ':', '#', '%', '.', '?', '+', '$', '=', "'", '{', '(', '_', ';', ',', '|', ']', '<', '/', '*', ')', '&', '^', '>', '@', '-', '}', '"', '!', '[', '\\', '~'}


In [17]:
df['post'] = df['post'].apply(elimin)
df['message'] = df['message'].apply(elimin)

df['post'] = df['post'].apply(clean)
df['message'] = df['message'].apply(clean)

Como resultado se creará la base final y se exportará a csv.

In [18]:
df = df.reset_index(drop=True)
del df['post']
df.to_csv('unlabeledTrainData.csv') # Guardamos la base en .csv

_________________________

In [19]:
 df = pd.read_csv('visitor_post_comunas.csv',index_col=None, header=0, encoding="latin1")

In [20]:
df.tail()

Unnamed: 0              created_time                                id  \
320         320  2018-04-10T02:30:28+0000  123364627704204_2133970523310261   
321         321  2018-04-05T19:45:43+0000  123364627704204_2127539987286648   
322         322  2018-04-04T12:54:12+0000  123364627704204_2125452507495396   
323         323  2018-04-04T02:37:37+0000  123364627704204_2124840417556605   
324         324  2018-04-02T12:24:13+0000  123364627704204_2122476437793003   

                                               message  \
320  https://www.facebook.com/MatiasLobosdirigente/...   
321  TALLER DE TEATRO:\n"ENTRENAMIENTO ACTORAL (ANT...   
322  Huerta a casa!!!!!!\nCanasta familiar de fruta...   
323  BUSCAS TRABAJO EN MEDELLIN\n\nCONDUCTOR SECTOR...   
324  De mi mayor consideraciÃ³n, me dirijo a Uds. a...   

                                                 story      comuna  
320  MatÃ­as Lobos shared a post to Comuna 8 de Bue...  bacomuna13  
321                                                NaN  bacomuna13  
322                                                NaN  bacomuna13  
323                                                NaN  bacomuna13  
324                                                NaN  bacomuna13

In [21]:
del df['Unnamed: 0'],df['id'],df['story']

In [22]:
df = df[df['message'].notna()]

Se procederá a la limpieza de las variables:

1- Se renombrará las comunas en la variable **comuna**.

2- Limpiaremos la variable **created_time** y la volveremos una variable fecha, con sus propiedades.

3- A la variable **message** se trabajará con todo aquello sea ruido, como por ejemplo la acentuación y la ñ que es leída como un caracter no comprendido por python.

** comuna **

In [23]:
df.loc[(df['comuna']=='bacomuna1'),'comuna']= 'ba_comuna1'
df.loc[(df['comuna']=='bacomuna2'),'comuna']= 'ba_comuna2'
df.loc[(df['comuna']=='bacomuna3'),'comuna']= 'ba_comuna3'
df.loc[(df['comuna']=='bacomuna4'),'comuna']= 'ba_comuna5'
df.loc[(df['comuna']=='bacomuna5'),'comuna']= 'ba_comuna6'
df.loc[(df['comuna']=='bacomuna6'),'comuna']= 'ba_comuna7'
df.loc[(df['comuna']=='bacomuna7'),'comuna']= 'ba_comuna8'
df.loc[(df['comuna']=='bacomuna8'),'comuna']= 'ba_comuna9'
df.loc[(df['comuna']=='bacomuna9'),'comuna']= 'ba_comuna10'
df.loc[(df['comuna']=='bacomuna10'),'comuna']= 'ba_comuna11'
df.loc[(df['comuna']=='bacomuna11'),'comuna']= 'ba_comuna12'
df.loc[(df['comuna']=='bacomuna12'),'comuna']= 'ba_comuna13'
df.loc[(df['comuna']=='bacomuna13'),'comuna']= 'ba_comuna15'

** created_time **

In [24]:
df['date'] = pd.to_datetime(df['created_time'])
del df['created_time']

** message **

Primero volveremos todo a minúscula los textos.

In [25]:
df = df.apply(lambda x: x.astype(str).str.lower())

Caracteres propios del español

In [26]:
pd.set_option('max_colwidth',20000)
df

message  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               este pobre peaton pone en juego su seguridad para cruzar, donde estan los controles de espacios publicos y seguridad. sres. para cuando el cambio.   
4                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [27]:
#a
df['message'] = df['message'].str.replace(u"ã¡", "a")
df['message'] = df['message'].str.replace('á', "a")

#e
df['message'] = df['message'].str.replace(u"ã©", "e")
df['message'] = df['message'].str.replace('é', "e")

#i
df['message'] = df['message'].str.replace(u"ã­", "i")
df['message'] = df['message'].str.replace('í', "i")

#o
df['message'] = df['message'].str.replace(u"ã³", "o")
df['message'] = df['message'].str.replace('ó', "o")

#u
df['message'] = df['message'].str.replace(u"ãº", "u")
df['message'] = df['message'].str.replace('ú', "u")

#ñ
df['message'] = df['message'].str.replace(u"ã±","ñ")

In [28]:
df = df.reset_index(drop=True)
df.to_csv('visitor_post_comun.csv') # Guardamos la base en .csv

Se observa que existe páginas web declaradas en el post y en comentarios. Se procederá a eliminar la ruta.

In [29]:
# Página web

df['message'] = df['message'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

Se eliminará todos los caracteres que no son ni texto ni número.

In [30]:
df['message'] = df['message'].apply(elimin)
df['message'] = df['message'].apply(clean)

In [31]:
df.head()

message  \
0                                                                                                                    este pobre peaton pone en juego su seguridad para cruzar donde estan los controles de espacios publicos y seguridad sres para cuando el cambio   
1                                                                                                                                                 esto esta ocurriendo en la esquina de charcas y billinguhrs en este momento e comuna y los controles bien gracias   
2                                                                                                                                                  taller de teatro en fermin espacio cultural todos los jueves de a h cobramos por clase estamos en culpina venite   
3                                                                                                                                                                      hola querria saber como hay que hacer para asistir a la visitas guiadas que realizan gracias   
4  fijate como este sinverguenza cobra estacionamiento donde no debe serum que algun sinverguenza como el lo apaa sres de comuna o transito cuando actuaran para que los contribuyentes veamos el cambio esto sigue ocurriendo en billinghurt al de ciudad autonoma   

       comuna                 date  
0  ba_comuna1  2018-06-22 15:03:22  
1  ba_comuna1  2018-06-22 14:54:07  
2  ba_comuna1  2018-06-08 18:37:21  
3  ba_comuna1  2018-06-07 20:39:10  
4  ba_comuna1  2018-06-07 14:44:39

In [32]:
df = df.reset_index(drop=True)
df.to_csv('visitor_post_comunas_clean.csv') # Guardamos la base en .csv

In [33]:
df['sentimiento'] = ""
df['label'] = ""
df['categoria'] = ""

In [34]:
df.to_csv('data.csv') # Guardamos la base en .csv

_______________________

In [35]:
df_label = pd.read_csv('dataLabeled.csv', header=0, index_col='Unnamed: 0', encoding="latin1")

In [36]:
df_labeled = df_label[df_label['message'].notna()]

In [37]:
df_labeled['label'].value_counts()

3    212
1     45
2     21
Name: label, dtype: int64

In [38]:
df_labeled['message'] = df_labeled['message'].apply(elimin)
df_labeled['message'] = df_labeled['message'].apply(clean)

In [39]:
df_labeled.loc[(df_labeled['comuna']=='ba_comuna1'),'comuna']= 'comuna01'
df_labeled.loc[(df_labeled['comuna']=='ba_comuna2'),'comuna']= 'comuna02'
df_labeled.loc[(df_labeled['comuna']=='ba_comuna3'),'comuna']= 'comuna03'
df_labeled.loc[(df_labeled['comuna']=='ba_comuna5'),'comuna']= 'comuna05'
df_labeled.loc[(df_labeled['comuna']=='ba_comuna6'),'comuna']= 'comuna06'
df_labeled.loc[(df_labeled['comuna']=='ba_comuna7'),'comuna']= 'comuna07'
df_labeled.loc[(df_labeled['comuna']=='ba_comuna8'),'comuna']= 'comuna08'
df_labeled.loc[(df_labeled['comuna']=='ba_comuna9'),'comuna']= 'comuna09'
df_labeled.loc[(df_labeled['comuna']=='ba_comuna10'),'comuna']= 'comuna10'
df_labeled.loc[(df_labeled['comuna']=='ba_comuna11'),'comuna']= 'comuna11'
df_labeled.loc[(df_labeled['comuna']=='ba_comuna12'),'comuna']= 'comuna12'
df_labeled.loc[(df_labeled['comuna']=='ba_comuna13'),'comuna']= 'comuna13'
df_labeled.loc[(df_labeled['comuna']=='ba_comuna15'),'comuna']= 'comuna15'

In [40]:
df_labeled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 0 to 276
Data columns (total 6 columns):
message        278 non-null object
comuna         278 non-null object
date           278 non-null object
sentimiento    278 non-null int64
label          278 non-null int64
categoria      278 non-null int64
dtypes: int64(3), object(3)
memory usage: 15.2+ KB


In [41]:
df_labeled.head()

message  \
0                                                                                                                    este pobre peaton pone en juego su seguridad para cruzar donde estan los controles de espacios publicos y seguridad sres para cuando el cambio   
1                                                                                                                                                 esto esta ocurriendo en la esquina de charcas y billinguhrs en este momento e comuna y los controles bien gracias   
2                                                                                                                                                  taller de teatro en fermin espacio cultural todos los jueves de a h cobramos por clase estamos en culpina venite   
3                                                                                                                                                                      hola querria saber como hay que hacer para asistir a la visitas guiadas que realizan gracias   
4  fijate como este sinverguenza cobra estacionamiento donde no debe serum que algun sinverguenza como el lo apaa sres de comuna o transito cuando actuaran para que los contribuyentes veamos el cambio esto sigue ocurriendo en billinghurt al de ciudad autonoma   

     comuna                 date  sentimiento  label  categoria  
0  comuna01  2018-06-22 15:03:22            1      3          8  
1  comuna01  2018-06-22 14:54:07            1      3          8  
2  comuna01  2018-06-08 18:37:21            2      1          4  
3  comuna01  2018-06-07 20:37:10            2      2          4  
4  comuna01  2018-06-07 14:44:37            1      3          7

Reemplazo los valores

In [42]:
df_labeled['sentimiento'].replace(1,'negativo', inplace=True)
df_labeled['sentimiento'].replace(2,'neutro', inplace=True)
df_labeled['sentimiento'].replace(3,'positivo', inplace=True)

df_labeled['label'].replace(1,'propuesta', inplace=True)
df_labeled['label'].replace(2,'consulta', inplace=True)
df_labeled['label'].replace(3,'reclamo', inplace=True)

df_labeled['categoria'].replace(1,'abl', inplace=True)
df_labeled['categoria'].replace(2,'mascotas', inplace=True)
df_labeled['categoria'].replace(3,'situacion_de_calle', inplace=True)
df_labeled['categoria'].replace(4,'cultura', inplace=True)
df_labeled['categoria'].replace(5,'gobierno', inplace=True)
df_labeled['categoria'].replace(6,'plazas', inplace=True)
df_labeled['categoria'].replace(7,'transito', inplace=True)
df_labeled['categoria'].replace(8,'seguridad', inplace=True)

In [43]:
df_labeled['message'].replace('eleccinjueves','eleccion jueves', inplace=True)

In [44]:
df_labeled_clean = df_labeled[['date','comuna','message','sentimiento','label','categoria']]
df_labeled_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278 entries, 0 to 276
Data columns (total 6 columns):
date           278 non-null object
comuna         278 non-null object
message        278 non-null object
sentimiento    278 non-null object
label          278 non-null object
categoria      278 non-null object
dtypes: object(6)
memory usage: 15.2+ KB


In [45]:
df_labeled_clean.head()

date    comuna  \
0  2018-06-22 15:03:22  comuna01   
1  2018-06-22 14:54:07  comuna01   
2  2018-06-08 18:37:21  comuna01   
3  2018-06-07 20:37:10  comuna01   
4  2018-06-07 14:44:37  comuna01   

                                                                                                                                                                                                                                                            message  \
0                                                                                                                    este pobre peaton pone en juego su seguridad para cruzar donde estan los controles de espacios publicos y seguridad sres para cuando el cambio   
1                                                                                                                                                 esto esta ocurriendo en la esquina de charcas y billinguhrs en este momento e comuna y los controles bien gracias   
2                                                                                                                                                  taller de teatro en fermin espacio cultural todos los jueves de a h cobramos por clase estamos en culpina venite   
3                                                                                                                                                                      hola querria saber como hay que hacer para asistir a la visitas guiadas que realizan gracias   
4  fijate como este sinverguenza cobra estacionamiento donde no debe serum que algun sinverguenza como el lo apaa sres de comuna o transito cuando actuaran para que los contribuyentes veamos el cambio esto sigue ocurriendo en billinghurt al de ciudad autonoma   

  sentimiento      label  categoria  
0    negativo    reclamo  seguridad  
1    negativo    reclamo  seguridad  
2      neutro  propuesta    cultura  
3      neutro   consulta    cultura  
4    negativo    reclamo   transito

In [46]:
#a
df_labeled_clean['message'] = df_labeled_clean['message'].str.replace(u"ã¡", "a")
df_labeled_clean['message'] = df_labeled_clean['message'].str.replace('á', "a")

#e
df_labeled_clean['message'] = df_labeled_clean['message'].str.replace(u"ã©", "e")
df_labeled_clean['message'] = df_labeled_clean['message'].str.replace('é', "e")

#i
df_labeled_clean['message'] = df_labeled_clean['message'].str.replace(u"ã­", "i")
df_labeled_clean['message'] = df_labeled_clean['message'].str.replace('í', "i")

#o
df_labeled_clean['message'] = df_labeled_clean['message'].str.replace(u"ã³", "o")
df_labeled_clean['message'] = df_labeled_clean['message'].str.replace('ó', "o")

#u
df_labeled_clean['message'] = df_labeled_clean['message'].str.replace(u"ãº", "u")
df_labeled_clean['message'] = df_labeled_clean['message'].str.replace('ú', "u")

#ñ
df_labeled_clean['message'] = df_labeled_clean['message'].str.replace(u"ã±","ñ")
df_labeled_clean['categoria'] = df_labeled_clean['categoria'].str.replace(u"Â±","ñ")

In [47]:
df_labeled_clean.to_csv('data_labeled.csv') # Guardamos la base en .csv